In [2]:
print("Installation de transformateurs.")
! pip install transformers --q
! pip install sentencepiece --q

Installation de transformateurs.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s eta 0:00:00


In [3]:
print("Importation des packages requis.")
from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering, MBartForConditionalGeneration, MBart50TokenizerFast
import pandas as pd
import numpy as np
import tensorflow as tf

Importation des packages requis.


In [4]:
print("Initialisation du modèle.")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")
transla_tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
transla_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

Initialisation du modèle.


All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [10]:
def translate(text, src_lang, tgt_lang):
    # Set the source and target languages for the tokenizer
    transla_tokenizer.src_lang = src_lang
    transla_tokenizer.tgt_lang = tgt_lang

    # Encode the input text
    encoded_text = transla_tokenizer(text, return_tensors="pt")

    # Generate the translation
    generated_tokens = transla_model.generate(**encoded_text, forced_bos_token_id=transla_tokenizer.lang_code_to_id[tgt_lang])

    # Decode the generated translation and skip special tokens
    translation = transla_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    return translation

In [11]:
print("Welcome to OUIOUIBAGUETTE. Your context-based, and also unfortunately French QnA Assistant :)")
print("As a French assistant, I really do not care if you know French or not.\nThis is the final English sentence you'll see from me.")

Welcome to OUIOUIBAGUETTE. Your context-based, and also unfortunately French QnA Assistant :)
As a French assistant, I really do not care if you know French or not.
This is the final English sentence you'll see from me.


In [13]:
stopnow = False
while not stopnow:
    context = input("Entrez le contexte: ")
    stopques = False
    while not stopques:
        question = input("Entrez votre question: ")
        inputs = tokenizer(question, context, return_tensors="tf")
        outputs = model(**inputs)

        answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
        answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
        predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
        answer = tokenizer.decode(predict_answer_tokens)
        src_lang = "en_XX"
        tgt_lang = "fr_XX"
        translated_sentence = translate(answer, src_lang, tgt_lang)
        print(translated_sentence[0])
        choice = input("Voulez-vous poser d'autres questions? [y/n]:")
        if choice == 'n':
            stopques = True
    choice2 = input("Souhaitez-vous utiliser un autre contexte pour les questions? [y/n]:")
    if choice2 == 'n':
        stopnow = True
        print("Merci de m'avoir utilisé!")

Entrez le contexte: I am Raajan. My name is derived from the Sanskrit term for the word king. I know how to speak English. I don't know how to speak French and Urdu. I love playing football. I really enjoy badminton.
Entrez votre question: What do I enjoy
badminton
Voulez-vous poser d'autres questions? [y/n]:y
Entrez votre question: What I don't know?
en français et en Urdu
Voulez-vous poser d'autres questions? [y/n]:y
Entrez votre question: My name is derived from?
terme sanscrit pour le mot roi
Voulez-vous poser d'autres questions? [y/n]:n
Souhaitez-vous utiliser un autre contexte pour les questions? [y/n]:y
Entrez le contexte: In the year 2022, the global population reached 7.9 billion people, marking a significant increase from the previous decade. The world's economy grew by 5.2%, driven by advances in technology and trade. The United Nations reported that 12% of the global population still lacked access to clean drinking water. In terms of education, the literacy rate rose to 87%